# MAE 269: HW3

In [1]:
import numpy as np
import sympy as sp
import scipy as sci
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
from scipy.integrate import solve_ivp
from scipy.optimize import fixed_point
from scipy.optimize import root
from scipy.optimize import brentq

## 1. Simulating Dynamical Systems

### Defining Numerical Functions

In [2]:
def pendulum(t,x,arg):
    x1, x2 = x;
    g, l = arg;
    return np.array([x2, -g*l*np.sin(x1)])

def VanDerPol(t,x,arg):
    x1, x2 = x;
    k, mu = arg;
    return np.array([x2, -k*x1 + mu*(1-np.power(x1,2))*x2])

def Hopf(t,x,arg):
    x1, x2 = x;
    w, mu = arg;
    return np.array([(mu-(np.power(x1,2)+np.power(x2,2)))*x1+w*x2,(mu-(np.power(x1,2)+np.power(x2,2)))*x2-w*x1])

def sfo(t,x,arg):
    x1, x2 = x;
    F = ((x1<=0)*2-1)*10
    eps = arg;
    return np.array([x2, F-2*eps*x2-x1])

### Defining function parameters

In [3]:
g = 9.81;
l = 1;
arg1 = [g,l]

k = 2;
mu = -1
arg2 = [k,mu]

w = 1;
mu = 1;
arg3 = [w,mu]

eps = 0.2
arg4 = eps

### Defining Symbolic Functions

In [4]:
x1, x2 = sp.symbols('x_1 x_2')

f1 = sp.Matrix([x2, -g*l*sp.sin(x1)])
f2 = sp.Matrix([x2, -k*x1 + mu*(1-x1**2)*x2])
f3 = sp.Matrix([(mu-(x1**2+x2**2))*x1+w*x2,(mu-(x1**2+x2**2))*x2+w*x1])
f4a = sp.Matrix([x2, 10-2*eps*x2-x1])
f4b = sp.Matrix([x2, -10-2*eps*x2-x1])

### Simulation Domain

In [5]:
t = np.linspace(0,10,100);

### Finding Nodes

In [20]:
p1 = sp.solve(f1,(x1,x2)).eval()
p2 = sp.solve(f2,(x1,x2)).feval()
p3 = sp.solve(f3,(x1,x2)).feval()
p4a = sp.solve(f4a,(x1,x2)).feval()
p4b = sp.solve(f4b,(x1,x2)).feval()

AttributeError: 'list' object has no attribute 'eval'

### Node Behavior

In [9]:
print("Pendulum:",end = " ")
print(f1.jacobian((x1,x2)).subs([(x1,p1[0]),(x2,p1[1])]).evalf().eigenvals())
print("-> Center at (0,0)")
print("-> Saddle at (pi,0)")

print()

print("VanDerPol:",end = " ")  
print(f2.jacobian((x1,x2)).subs([(x1,p2[0]),(x2,p2[1])]).evalf().eigenvals())
print("-> Repelling Focus at (0,0)")
print()

print("Hopf:",end = " ")  
print(f3.jacobian((x1,x2)).subs([(x1,p3[0]),(x2,p3[1])]).evalf().eigenvals())
print("-> Unstable Node at (0,0)")
print()

print("STF:",end = " ")  
print(f4.jacobian((x1,x2)).subs([(x1,p4[0]),(x2,p4[1])]).evalf().eigenvals())
print("-> Attracting Focus at (0,0)")
print()

Pendulum: {-3.13209195267317*I: 1, 3.13209195267317*I: 1}
-> Center at (0,0)
-> Saddle at (pi,0)

VanDerPol: {0.5 - 1.3228756555323*I: 1, 0.5 + 1.3228756555323*I: 1}
-> Repelling Focus at (0,0)

Hopf: {0: 1, 2.00000000000000: 1}
-> Unstable Node at (0,0)

STF: {-0.2 - 0.979795897113271*I: 1, -0.2 + 0.979795897113271*I: 1}
-> Attracting Focus at (0,0)



In [54]:
def plot_system(f,arg,T,xmin,xmax,ymin,ymax,t,fig):
    x = np.linspace(xmin,xmax,19)
    y = np.linspace(ymin,ymax,19);
    X, Y = np.meshgrid(x,y)
    XY = [X, Y]

    UV = f(t,XY,arg)
    U, V = UV
    Uhat = U * (1 / np.sqrt(np.power(U,2) + np.power(V,2)))
    Vhat = V * (1 / np.sqrt(np.power(U,2) + np.power(V,2)))


    h = 0.01 # step size (1 ms)
    tf = 10 # duration (1 s)
    N = int(tf/h);

    s1 = solve_ivp(f, [0, 10],T[0], args=[arg], t_eval=t).y
    s2 = solve_ivp(f, [0, 10],T[1], args=[arg], t_eval=t).y
    s3 = solve_ivp(f, [0, 10],T[2], args=[arg], t_eval=t).y

    gs0 = gridspec.GridSpec(1, 1, figure=fig)

    gs00 = gridspec.GridSpecFromSubplotSpec(2, 2, subplot_spec=gs0[0])

    ax1 = fig.add_subplot(gs00[:, :-1])
    ax2 = fig.add_subplot(gs00[-1, -1])
    ax3 = fig.add_subplot(gs00[-0, -1])
    fig.tight_layout(pad=1)

    ax1.quiver(X,Y,Uhat,Vhat)
    ax1.plot(s1[0,:],s1[1,:])
    ax1.plot(s2[0,:],s2[1,:])
    ax1.plot(s3[0,:],s3[1,:])
    ax1.set_xlim([xmin,xmax])
    ax1.set_ylim([ymin,ymax])
    ax2.plot(t,s1[0])
    ax2.plot(t,s2[0])
    ax2.plot(t,s3[0])
    ax3.plot(t,s1[1])
    ax3.plot(t,s2[1])
    ax3.plot(t,s3[1])
    return [ax1, ax2, ax3]

%matplotlib qt

In [57]:
plt.close('all')



fig1 = plt.figure(figsize=(12,4))
t1 = np.linspace(0,10,100);
ax11, ax12, ax13 = plot_system(pendulum,arg1,[[1,0],[1.5,0],[2,0]],-np.pi,np.pi,-10,10,t1,fig1);
eq1, = ax11.plot(p1[0][0].evalf(),p1[0][1].evalf(),'go',label='Center')
eq2, = ax11.plot(p1[1][0].evalf(),p1[1][1].evalf(),'rx',label='Saddle')
eq2, = ax11.plot(-p1[1][0].evalf(),p1[1][1].evalf(),'rx',label='Saddle')
ax11.legend(handles=[eq1, eq2])

fig2 = plt.figure(figsize=(12,4))
t2 = np.linspace(0,10,100);
ax1, ax2, ax3 = plot_system(VanDerPol,arg2,[[1,0],[1.5,0],[2,0]],-np.pi,np.pi,-10,10,t2,fig2)
eq1, = ax21.plot(p1[0][0].evalf(),p1[0][1].evalf(),'go',label='Center')


fig3 = plt.figure(figsize=(12,4))
t3 = np.linspace(0,10,100);
plot_system(Hopf,arg3,[[-6,6],[-3,-3],[9,9]],-10,10,-10,10,t3,fig3)

fig4 = plt.figure(figsize=(12,4))
t4 = np.linspace(0,3,100);
plot_system(sfo,arg4,[[-1,0],[-2,-0],[-3,0]],-10,10,-10,10,t4,fig4)

/var/folders/9w/9hcqnn0s191_z131mzx0lmx40000gn/T/ipykernel_20085/1444858045.py:9: RuntimeWarning: divide by zero encountered in divide
  Uhat = U * (1 / np.sqrt(np.power(U,2) + np.power(V,2)))
/var/folders/9w/9hcqnn0s191_z131mzx0lmx40000gn/T/ipykernel_20085/1444858045.py:9: RuntimeWarning: invalid value encountered in multiply
  Uhat = U * (1 / np.sqrt(np.power(U,2) + np.power(V,2)))
/var/folders/9w/9hcqnn0s191_z131mzx0lmx40000gn/T/ipykernel_20085/1444858045.py:10: RuntimeWarning: divide by zero encountered in divide
  Vhat = V * (1 / np.sqrt(np.power(U,2) + np.power(V,2)))
/var/folders/9w/9hcqnn0s191_z131mzx0lmx40000gn/T/ipykernel_20085/1444858045.py:10: RuntimeWarning: invalid value encountered in multiply
  Vhat = V * (1 / np.sqrt(np.power(U,2) + np.power(V,2)))


[<Axes: >, <Axes: >, <Axes: >]

In [21]:
p1

[(0.0, 0.0), (3.14159265358979, 0.0)]

In [47]:
type(p1[0][0])

sympy.core.numbers.Float

In [40]:
p1[:][1]

(3.14159265358979, 0.0)

In [50]:
p1[0][0].evalf()

0